## Feature Engineering
Documentation: https://support.google.com/analytics/answer/3437719?hl=en

Note: the field 'unique_visit_id' should be unique but is not (around 1800 repeated unique_visit_id's):

trainCSV['unique_visit_id'] = trainCSV.fullVisitorId +'_'+ trainCSV.visitId.apply(str)

In [1]:
import datetime as dt
import numpy as np
from data_wrangling import *
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 150)

In [2]:
%%time
trainCSV = pd.read_csv('train_v2_flat.csv', sep=',', encoding='utf-8', low_memory=False).drop(columns=['trafficSource_campaignCode'])
trainCSV = fill_empty_values(trainCSV)

CPU times: user 48.1 s, sys: 4.24 s, total: 52.4 s
Wall time: 55 s


In [3]:
cool_columns = ['fullVisitorId', 'date', 'geoNetwork_city', 'geoNetwork_country', 'device_browser','visits', 'paying_visits', 'totals_transactionRevenue']
trainCSV = trainCSV[cool_columns]

start_train_visits = trainCSV.date.min()
end_train_visits = trainCSV.date.max() - timedelta(days=276)

train_visits = trainCSV[(trainCSV.date >= start_train_visits) & (trainCSV.date<end_train_visits)]
validation_visits = trainCSV[(trainCSV.date>= end_train_visits)]

In [4]:
%%time
train = get_training_data_from_visits(train_visits)
validation = get_training_data_from_visits(validation_visits)

CPU times: user 32.9 s, sys: 1.81 s, total: 34.7 s
Wall time: 35.2 s


In [5]:
%%time
train_e = label_encode_object_dtypes(encoder_trainer = train, df_to_encode = train)
validation_e = label_encode_object_dtypes(encoder_trainer = train, df_to_encode = validation)

CPU times: user 1min 29s, sys: 1.15 s, total: 1min 30s
Wall time: 1min 31s


In [6]:
train_e.to_csv("train_cool_columns.csv", index=False)
validation_e.to_csv("validation_cool_columns.csv", index=False)